# 소개: 허깅페이스를 활용한 대형 언어 모델 이해하기

## 개요

대형 언어 모델(Large Language Models, LLMs)은 인간과 유사한 텍스트를 이해하고 생성함으로써 자연어 처리(Natural Language Processing, NLP)를 혁신화하는 고급 딥 러닝 모델입니다. 이 튜토리얼에서는 HuggingFace 프레임워크를 사용하여 텍스트 생성을 위해 설계된 **Llama-2-Ko**의 기본 모델에 대한 기본 원리를 탐구합니다.

### 주요 요점:

- **정의:** LLMs는 신경망을 활용하여 텍스트를 이해하고 생성하며, 방대한 규모의 훈련 데이터셋을 통해 언어 패턴, 맥락, 의미를 파악합니다.

- **응용:** LLMs는 텍스트 생성, 요약, 번역, 질문 응답 및 감정 분석과 같은 다양한 NLP 작업에 활용됩니다.

## 텍스트 생성 모델

LLMs의 일반적인 아키텍처인 텍스트 생성 모델은 일관된 및 문맥적으로 관련된 텍스트를 생성하는 데 특화되어 있습니다. 이들은 딥 러닝 기술을 활용하여 언어의 복잡성을 이해하고 인간과 유사한 응답을 생성합니다.

### GPT 모델:

- **아키텍처:** GPT 모델은 트랜스포머 아키텍처를 사용하여 어텐션 메커니즘, 셀프 어텐션 레이어 및 위치 인코딩을 통해 효과적으로 장거리 종속성과 문맥 정보를 캡처합니다.

- **작동 원리:** 대량의 텍스트 데이터에서 **비지도 학습 방식**으로 훈련된 GPT 모델은 시퀀스에서 **다음 단어를 예측**하도록 학습되어 언어 패턴을 풍부하게 이해합니다.

이 튜토리얼에서는 명시적으로 지시사항이나 채팅 상호작용에 특화되지 않은 **Llama-2-Ko의 기본 모델** 작업에 중점을 둡니다. 이후의 튜토리얼에서는 **지시사항 튜닝 및 채팅 튜닝 LLMs**의 복잡성을 탐구할 것입니다. 이 기본 모델을 사용하여 챗봇을 만드는 실제 측면으로 넘어가 봅시다.
s.t generation models.


# 실용 가이드: HuggingFace를 사용한 텍스트 생성

이 섹션에서는 Hugging Face를 사용한 텍스트 생성의 실용적인 단계를 안내합니다. 일관된 및 문맥적으로 관련된 텍스트를 생성하기 위해 GPT 모델을 활용하는 데 중점을 둘 것입니다. 코드 스니펫을 따라가면 구현 세부 사항을 이해하는 데 도움이 될 것입니다.

## 단계 1: HuggingFace Transformers 라이브러리 설치

시작하려면 Hugging Face Transformers 라이브러리를 설치해야 합니다. 이는 자연어 처리 작업을 위한 포괄적인 툴킷입니다. 다음 명령어를 코드 셀에서 실행하여 라이브러리를 설치하세요


In [1]:
%%capture
!pip install transformers

## 단계 2: 필요한 라이브러리 가져오기

Hugging Face의 강력한 Transformers 라이브러리를 사용하려면 필수 구성 요소를 가져와야 합니다. 특히 `AutoModelForCausalLM` 및 `AutoTokenizer` 두 핵심 클래스를 사용할 것입니.


### AutoModelForCausalLM

`AutoModelForCausalLM`은 사전 훈련된 인과 언어 모델을 로드하기 위한 간단한 인터페이스를 제공하는 클래스입니다. 인과 언어 모델은 순서가 중요한 자동 회귀 작업에 적합하며 텍스트 생성과 같은 작업에 사용됩니다. 이 클래스는 제공된 모델 이름을 기반으로 적절한 모델 아키텍처를 자동으로 식별하여 코드를 변경하지 않고 다양한 모델을 실험할 수 있도록 합니다.

튜토리얼의 맥락에서 우리는 이 클래스를 사용하여 텍스트 생성을 위해 사전 훈련된 GPT (Generative Pre-trained Transformer) 모델을 로드합니다. `from_pretrained` 메서드는 모델 가중치를 로드하며 `AutoModelForCausalLM` 클래스는 특정 아키텍처와의 호환성을 보장합니다.

### AutoTokenizer

`AutoTokenizer`는 Hugging Face의 Transformers 라이브러리의 또 다른 중요한 구성 요소입니다. 토크나이저는 원시 텍스트를 모델에 공급할 수 있는 형식으로 처리하는 데 필수적입니다. `AutoTokenizer` 클래스는 제공된 모델 이름을 기반으로 적절한 토크나이저를 자동으로 선택합니다.

우리의 튜토리얼에서는 선택한 GPT 모델에 해당하는 토크나이저를 로드하는 데 사용합니다. 이 토크나이저는 입력 텍스트를 토큰으로 분해하며, 이 토큰은 그 후 모델에 의해 처리됩니다. `from_pretrained` 메서드는 선택한 GPT 모델과 관련된 사전 훈련된 토크나이저를 로드합니다.

`AutoModelForCausalLM` 및 `AutoTokenizer`를 사용하여 각 모델의 아키텍처나 토큰화 프로세스의 구체적인 내용을 걱정하지 않고도 다양한 사전 훈련된 언어 모델과 쉽게 작업할 수 있습니다.

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

/home/ubuntu/miniforge3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 단계 3: 사전 훈련된 GPT 모델 로드

이제 텍스트 생성을 위해 사전 훈련된 GPT 모델을 로드해 봅시다. 이 튜토리얼에서는 한국어 텍스트 생성을 위해 특별히 제작된 "beomi/llama-2-ko-7b" 모델을 사용합니다. 모델에 대한 간단한 개요는 다음과 같습니다:

### Llama-2-Ko 개요

- **개발자:** 이준범 (Beomi)
- **변형:** Llama-2-Ko는 7B (이 튜토리얼에서 사용됨), 13B, 그리고 70B와 같은 다양한 매개변수 크기 및 사전 훈련 및 미세 조정된 변형을 제공합니다.
- **입력 및 출력:** 모델은 입력 텍스트를 받아들이고 텍스트를 생성하여 출력으로 생성하므로 다양한 텍스트 생성 작업에 적합합니다.

### 어휘 확장

- **원본 Llama-2:** Sentencepiece BPE를 사용한 32,000 크기의 어휘.
- **확장된 Llama-2-Ko:** 한국어 어휘 및 병합을 통한 어휘 크기 증가로 46,336 크기의 어휘.


In [3]:
# HuggingFace 허브에서 모델 이름 초기화
model_name = "beomi/llama-2-ko-7b"

# 모델 및 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards: 100%|██████████| 15/15 [00:02<00:00,  5.45it/s]


## 단계 4: 입력 토큰화

토큰화는 자연어 처리에서 중요한 단계로, 텍스트 시퀀스를 토큰이라 불리는 작은 단위로 분해하는 과정입니다. 이러한 토큰은 언어 모델의 기본 구성 요소로 작용합니다.

Llama-2-Ko를 사용하여 문장 "안녕하세요, 오늘은 날씨가 좋네요."의 토큰화 예제를 살펴보겠습니다:


In [5]:
# 문장을 토큰화
tokens = tokenizer.tokenize("안녕하세요, 오늘은 날씨가 좋네요.") # 안녕하세요, 오늘은 날씨가 좋네요.
print("토큰화된 문장:", tokens)

# 문장을 PyTorch Tensor로 인코딩하여 토큰화
tokens_pt = tokenizer("안녕하세요, 오늘은 날씨가 좋네요.", return_tensors="pt")
print("인코딩된 토큰", tokens_pt['input_ids'])

토큰화된 문장: ['▁안녕', '하세요', ',', '▁오늘은', '▁날', '씨가', '▁좋네요', '.']
인코딩된 토큰 tensor([[    1, 43116, 33055, 29892, 41636, 32502, 33876, 32864, 29889]])


이 코드 스니펫에서:

- `tokenizer.tokenize("안녕하세요, 오늘은 날씨가 좋네요.")`: 입력 문장을 토큰 목록으로 토큰화합니다. 결과 목록인 tokens은 입력이 어떻게 개별 단위로 분해되었는지를 나타냅니다.

- `tokenizer("안녕하세요, 오늘은 날씨가 좋네요.", return_tensors="pt")`: 입력 문장을 토큰화하고 입력 ID를 포함하는 PyTorch Tensor를 반환합니다. `tokens_pt['input_ids']`를 사용하여 액세스하는 이 Tensor는 토큰의 숫자적 표현을 제공합니다. 이는 Llama-2-Ko 모델에 입력하기에 적합한 형식입니다.


## 단계 5: 텍스트 생성 (다음 토큰 예측)

이제 입력을 토큰화했으니, Llama-2-Ko 모델을 사용하여 텍스트를 생성해봅시다. 이 단계에서는 모델이 입력 토큰을 기반으로 시퀀스의 다음 토큰을 예측하는 다음 토큰 예측을 수행합니.


In [6]:
# 다음 토큰 예측을 사용하여 텍스트 생성
output = model.generate(input_ids=tokens_pt['input_ids'], max_length=100,)
print("생성된 출력 (ID):", output)

# 생성된 출력 디코딩
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("생성된 텍스트:", generated_text)


생성된 출력 (ID): tensor([[    1, 43116, 33055, 29892, 41636, 32502, 33876, 32864, 29889, 29871,
         30166, 35715, 31354, 34333, 35874, 32500, 31286, 34595, 31435, 35362,
         32349, 32869, 29889, 29871, 30166, 34120,   525, 33139, 32019, 30906,
         32140, 32921, 32305, 29915, 32295, 32500, 32214, 29889, 29871, 30166,
         30393, 32500, 31354, 34333, 29871, 29906, 29900, 29896, 29953, 31571,
         29871, 29896, 36648, 34156, 34537, 32500, 32214, 29889, 29871, 30166,
         31607, 33070, 31054, 35323, 29871, 29906, 29900, 30890, 34685, 43505,
         29892, 29871, 30166, 44897, 31299, 33787, 31286, 33889, 32504, 43143,
         33406, 33115, 32757, 32043, 29889, 29871, 30166, 31607, 33070, 31054,
         35323,   525, 35356, 32153, 32624, 39486, 32474, 17901, 32295, 34963]])
생성된 텍스트: 안녕하세요, 오늘은 날씨가 좋네요. ​오늘은 제가 좋아하는 책을 소개해드리려고 합니다. ​바로 '나는 나로 살기로 했다'라는 책입니다. ​이 책은 제가 2016년 1월에 읽었던 책입니다. ​그 당시에 저는 20대 후반이었고, ​직장생활을 하면서 많은 스트레스를 받고 있었습니다. ​그 당시에 저는 '내가 왜 이렇게 살아야 하지?'라는

이 코드 스니펫에서:

- `output = model.generate(input_ids=tokens_pt['input_ids'], max_length=50)`: Llama-2-Ko 모델의 `generate` 메서드를 사용하여 시퀀스의 다음 토큰을 예측합니다. `input_ids` 매개변수는 토큰화에서 얻은 입력 ID를 포함하는 PyTorch Tensor로 설정됩니다 (`tokens_pt['input_ids']`). `max_length` 매개변수는 생성된 시퀀스의 최대 길이를 제어합니다.

- `generated_text = tokenizer.decode(output[0], skip_special_tokens=True)`: 출력 ID를 생성한 후, 이 라인은 토크나이저의 `decode` 메서드를 사용하여 숫자 출력을 인간이 읽을 수 있는 텍스트로 변환합니다. `output[0]`는 생성된 출력의 첫 번째 시퀀스에 해당하며, `skip_special_tokens=True`는 최종 디코딩된 텍스트에서 특수 토큰을 제거니다.


# 마무리 노트

이 튜토리얼에서는 **대규모 언어 모델 (LLMs)**의 매혹적인 세계를 탐험하면서 **Llama-2-Ko 기본 모델**을 사용한 텍스트 생성에 중점을 두었습니다. 이 **기본 모델**은 주로 텍스트를 예측하고 완성하는 데 훈련되었으며 명시적으로 지시 따르기나 채팅 기반 상호 작용을 위해 튜닝되지 않았다는 것을 이해하는 것이 중요합니다.

이 기본 모델의 행동을 설명하기 위한 예제를 간단히 살펴보겠니다:


In [7]:
# 기본 모델을 사용한 예제
prompt = "인도네시아의 도시들을 인구 규모별로 정렬된 목록으로 제시해주세요." # Give me a list of the cities in Indonesia sorted by population size.

# 기본 모델을 사용하여 텍스트 생성
output = model.generate(input_ids=tokenizer(prompt, return_tensors="pt")['input_ids'], max_length=100,)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print("Prompt:", prompt)
print("생성된 텍스트:", generated_text)

Prompt: 인도네시아의 도시들을 인구 규모별로 정렬된 목록으로 제시해주세요.
생성된 텍스트: 인도네시아의 도시들을 인구 규모별로 정렬된 목록으로 제시해주세요.⊙ 1960년대 1960년 4.19혁명으로 이승만 독재정권이 무너지고 장면 민주당 정권이 들어섰으나 1961년 5.16 군사정변으로 박정희 군사정권이 들어섰다. 1962년 1월 1일 시행된 '향토예비군 설치법'에 따라 19


**참고:** 위의 예제에서 볼 수 있듯이, 기본 모델은 프롬프트를 기반으로 텍스트를 생성하지만 주어진 지시사항을 엄격히 따르지 않을 수 있습니다.

다음 튜토리얼에서는 특별히 튜닝된 LLMs의 특수 기능을 탐구하여 지시를 따르고 채팅과 유사한 상호 작용을 수행하는 능력을 알아볼 것입니다. 이러한 고급 언어 모델의 특별한 기능에 대한 깊이 있는 여정을 기대해주세!
